In [1]:

import numpy as np

# Load the dataset
test = np.loadtxt('test.csv', delimiter=',')
train = np.loadtxt('train.csv', delimiter=',')
# split both sets into X (input / data) and Y (output / lables) parts
data = np.array(train)[:,0:2]
labels = np.array(train)[:,2]

d_test = np.array(test)[:,0:2]
l_test = np.array(test)[:,2]


In [2]:
#Declaring custom loss, activation and metrics

import keras.backend as K

def euclidean_distance_loss(y_true, y_pred):
    return K.sqrt(K.sum(K.square(y_pred - y_true), axis=-1))

# For custom metrics (this is mean squarred error)
def mean_pred(y_true, y_pred):
    return K.mean(np.square(np.subtract(y_true,y_pred)))

#custom activation function
from keras.layers import Activation
#from keras import backend as K
from keras.utils.generic_utils import get_custom_objects

def custom_activation(x):
    return  K.relu(x**3) 

get_custom_objects().update({'custom_activation': Activation(custom_activation)})


Using TensorFlow backend.


In [3]:
from keras.layers import Input, Dense, add, Activation
from keras.models import Model


#Actual DR Network
# Data manipulation and model settings
#number of neurons per layer
m=10
#padding zeros so that the input dimension can match our desired dimension:
data = np.hstack((data, np.zeros((data.shape[0], m - data.shape[1] ))))
data_test = np.hstack((d_test, np.zeros((d_test.shape[0], m - d_test.shape[1] ))))

number_of_hidden_layers_plus_input_layer = 4

#Declaring The Model

stack = []
inputTensor = Input(shape=(m,))

for i in range(number_of_hidden_layers_plus_input_layer):
    # First Layer Of the i-th Block 
    if(i==0):
        stack.append(Dense(m, input_dim = (None,m,),activation='custom_activation')(inputTensor))
    else: 
        stack.append(Dense(m, activation='custom_activation')(stack[len(stack)-1]))
    # Second Layer of the i-th Block
    #stack.append(Dense(m,activation='custom_activation')(stack[len(stack)-1]))
    # Third Layer of the i-th Block 
    # (adding the out put of the first layer and second layer of the block together)
    stack.append(Dense(m)(stack[len(stack)-1]))
    stack.append(add([stack[len(stack)-2],stack[len(stack)-1]]))
    stack.append(Activation('custom_activation')(stack[len(stack)-1]))

#adding the final Layer
finalOutput = Dense(1)(stack[len(stack)-1])


#Running The Model
model = Model(inputTensor,finalOutput)
#model.compile(optimizer='adam', loss=euclidean_distance_loss, metrics=[mean_pred])
model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['mae', 'acc'])
model.fit(data, labels, epochs=1, batch_size=10)  

loss, MAE, acc = model.evaluate(data_test, l_test)
print('Loss Funcion Minima: %.8f' % (loss))
print('Mean Absolute Error: %.8f' % (MAE))
print('Accuracy: %.8f' % (acc))
#print(model.metrics_names)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/1
694032/694032 [==============================] - 104s 150us/step
Loss Funcion Minima: 0.00267311
Mean Absolute Error: 0.04467424
Accuracy: 0.00000865


In [4]:
#making predictions and save it!

p = model.predict(data_test)
out = np.hstack((d_test, p))

np.savetxt('gnuformatted_prediction.csv', train, delimiter=',')

# 

In [5]:
# Open the file
with open( 'report.txt','w') as fh:
    # Pass the file handle in as a lambda function to make it callable
    model.summary(print_fn=lambda x: fh.write(x + '\n'))
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 10)           0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 10)           110         input_1[0][0]                    
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 10)           110         dense_1[0][0]                    
__________________________________________________________________________________________________
add_1 (Add)                     (None, 10)           0           dense_1[0][0]                    
                                                                 dense_2[0][0]                    
__________

In [6]:
model

In [12]:
model.output.consumers

<bound method Tensor.consumers of <tf.Tensor 'dense_9/BiasAdd:0' shape=(?, 1) dtype=float32>>